In [17]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tqdm as tqdm

import matplotlib.pyplot as plt

%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn


# special matplotlib argument for improved plots
from matplotlib import rcParams

In [18]:
data = pd.read_csv("MSN Historical Data.csv")
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2017/12/1,68500,67000,69400,66500,725370,0.02
1,2017/12/4,69000,70000,70000,68800,1190000,0.01
2,2017/12/5,68900,69000,70100,68600,777070,0.00
3,2017/12/6,67700,68900,69000,66000,1080000,-0.02
4,2017/12/7,69000,68600,69800,67300,1560000,0.02


In [19]:
dataset = data.loc[:,["Date","Price"]]
dataset.head()

,Date,Price
0,2017/12/1,68500
1,2017/12/4,69000
2,2017/12/5,68900
3,2017/12/6,67700
4,2017/12/7,69000


In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1249 non-null   object
 1   Price   1249 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 19.6+ KB


In [21]:
dataset.Date = pd.to_datetime(dataset.Date,format = "%Y/%m/%d")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1249 non-null   datetime64[ns]
 1   Price   1249 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 19.6 KB


In [22]:
dataset = dataset.rename(columns = {"Price" : "y"})
dataset = dataset.rename(columns = {"Date" : "ds"})
dataset.head()

,ds,y
0,2017-12-01,68500
1,2017-12-04,69000
2,2017-12-05,68900
3,2017-12-06,67700
4,2017-12-07,69000


In [23]:
test_days = 249
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail(5)

,ds,y
1244,2022-11-25,94000
1245,2022-11-28,97200
1246,2022-11-29,99000
1247,2022-11-30,102000
1248,2022-12-01,99000


In [24]:
print(len(training_set))
print(len(test_set))

1000
249


In [25]:
training_set.tail(5)

,ds,y
995,2021-11-26,130291
996,2021-11-29,126155
997,2021-11-30,123425
998,2021-12-01,124004
999,2021-12-02,125741


In [26]:
m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = 10,
            holidays_prior_scale = 10,
            changepoint_prior_scale = 0.05)
m.fit(training_set)

In [27]:
future = m.make_future_dataframe(periods = len(test_set), freq = "D")
future.tail()

,ds
1244,2022-08-04
1245,2022-08-05
1246,2022-08-06
1247,2022-08-07
1248,2022-08-08


In [28]:
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(5)

,ds,y
1244,2022-08-04,94000
1245,2022-08-05,97200
1246,2022-08-06,99000
1247,2022-08-07,102000
1248,2022-08-08,99000


In [29]:
forecast = m.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-12-01,9395.289431,65504.638139,73564.729209,9395.289431,9395.289431,6.425324,6.425324,6.425324,6.096344,6.096344,6.096344,0.328980,0.328980,0.328980,0.0,0.0,0.0,69763.065773
1,2017-12-04,9599.564064,66711.629889,74914.943801,9599.564064,9599.564064,6.363267,6.363267,6.363267,6.096688,6.096688,6.096688,0.266579,0.266579,0.266579,0.0,0.0,0.0,70684.149387
2,2017-12-05,9667.655609,66870.992392,75074.481242,9667.655609,9667.655609,6.331995,6.331995,6.331995,6.089114,6.089114,6.089114,0.242881,0.242881,0.242881,0.0,0.0,0.0,70883.202783
3,2017-12-06,9735.747154,66877.902005,75040.212225,9735.747154,9735.747154,6.310074,6.310074,6.310074,6.092395,6.092395,6.092395,0.217679,0.217679,0.217679,0.0,0.0,0.0,71169.036227
4,2017-12-07,9803.838698,67500.213271,75493.254686,9803.838698,9803.838698,6.285971,6.285971,6.285971,6.094964,6.094964,6.094964,0.191007,0.191007,0.191007,0.0,0.0,0.0,71430.485889


In [30]:
predictions_prophet = forecast.yhat[-test_days:].rename("prophet")
predictions_prophet[:5]

1000    122851.984284
1001   -232643.026337
1002   -233319.969638
1003    122117.501886
1004    121692.601577
Name: prophet, dtype: float64

In [31]:
from fbprophet.diagnostics import cross_validation
cs = cross_validation(m,
                         #Dự đoán 30 ngày tiếp theo
                         horizon = '30 days',
                         #Lấy 540 ngày dể đào tạo
                         initial = '540 days',
                         #Xử lý song song
                         parallel = "processes")

INFO:fbprophet:Making 60 forecasts with cutoffs between 2019-06-01 00:00:00 and 2021-11-02 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fbe78d90550>


In [32]:
y_true = cs['y'].values
y_pre = cs['yhat'].values

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
mae = mean_absolute_error(y_true,y_pre)
mape = mean_absolute_percentage_error(y_true,y_pre)
mse = mean_squared_error(y_true, y_pre)
rmse = np.sqrt(mse)
r2 = r2_score(y_true,y_pre)

print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape * 100:.2f}%")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2: {r2:.2f}")

MAE: 6398.54
MAPE: 9.41%
MSE: 74513380.30
RMSE: 8632.11
R2: 0.86


In [34]:
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': np.arange(5, 10, 5), #[5, 10]  np.arange(5, 20, 5),
              'changepoint_prior_scale': np.arange(0.01, 0.05, 0.01),  # [0.01, 0.02, 0.03, 0.04, 0.05]
              'holidays_prior_scale': np.arange(5, 10, 5)}#[5, 10]
grid = ParameterGrid(param_grid)
list(grid)[:3]  

[{'changepoint_prior_scale': 0.01,
  'holidays_prior_scale': 5,
  'seasonality_prior_scale': 5},
 {'changepoint_prior_scale': 0.02,
  'holidays_prior_scale': 5,
  'seasonality_prior_scale': 5},
 {'changepoint_prior_scale': 0.03,
  'holidays_prior_scale': 5,
  'seasonality_prior_scale': 5}]

In [ ]:
rmse = []
for params in grid:
  #build model
  m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = params['seasonality_prior_scale'],
            holidays_prior_scale = params['holidays_prior_scale'],
            changepoint_prior_scale = params['changepoint_prior_scale'])
  m.fit(training_set)

  #cross-validation
  cs = cross_validation(m,
                         horizon = '30 days',
                         initial = '540 days',
                         parallel = "processes")
  
  #gather the results
  error = np.sqrt(mean_squared_error(cs['y'], 
                                     cs['yhat']))
  rmse.append(error)

INFO:fbprophet:Making 60 forecasts with cutoffs between 2019-06-01 00:00:00 and 2021-11-02 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fbe78d6e340>
INFO:fbprophet:Making 60 forecasts with cutoffs between 2019-06-01 00:00:00 and 2021-11-02 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fbe7a742b50>
INFO:fbprophet:Making 60 forecasts with cutoffs between 2019-06-01 00:00:00 and 2021-11-02 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fbe78d34340>
INFO:fbprophet:Making 60 forecasts with cutoffs between 2019-06-01 00:00:00 and 2021-11-02 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fbe75b94760>


In [ ]:
best_params = grid[np.argmin(rmse)]
print(best_params)

{'seasonality_prior_scale': 5, 'holidays_prior_scale': 5, 'changepoint_prior_scale': 0.04}


In [ ]:
m = Prophet(growth = "linear",
          yearly_seasonality = True,
          weekly_seasonality = True,
          daily_seasonality = False,
          seasonality_mode = "multiplicative",
          seasonality_prior_scale = best_params['seasonality_prior_scale'],
          holidays_prior_scale = best_params['holidays_prior_scale'],
          changepoint_prior_scale = best_params['changepoint_prior_scale'])
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set),
                                 freq = "D")
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(1)

,ds,y
1248,2022-08-08,99000


In [ ]:
forecast = m.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-12-01,8445.385742,65725.989859,74235.011236,8445.385742,8445.385742,7.274099,7.274099,7.274099,6.873376,6.873376,6.873376,0.400722,0.400722,0.400722,0.0,0.0,0.0,69877.954208
1,2017-12-04,8635.025387,66391.863017,75187.561489,8635.025387,8635.025387,7.197170,7.197170,7.197170,6.874073,6.874073,6.874073,0.323096,0.323096,0.323096,0.0,0.0,0.0,70782.770647
2,2017-12-05,8698.238602,66806.801077,75418.021354,8698.238602,8698.238602,7.159561,7.159561,7.159561,6.865577,6.865577,6.865577,0.293984,0.293984,0.293984,0.0,0.0,0.0,70973.807176
3,2017-12-06,8761.451817,67078.103315,75641.525798,8761.451817,8761.451817,7.132503,7.132503,7.132503,6.869286,6.869286,6.869286,0.263217,0.263217,0.263217,0.0,0.0,0.0,71252.533782
4,2017-12-07,8824.665032,67201.216725,75467.169876,8824.665032,8824.665032,7.103268,7.103268,7.103268,6.872419,6.872419,6.872419,0.230849,0.230849,0.230849,0.0,0.0,0.0,71508.623757


In [ ]:
print(len(forecast['yhat']))
print(len(forecast.yhat[-test_days:]))
print(len(test_set["ds"]))

1249
249
249


In [ ]:
predictions_prophet_tuned = forecast.yhat[-test_days:].rename("prophet_tuned")
predictions_prophet_tuned.index = test_set["ds"]
predictions_prophet_tuned.head()

ds
2021-12-03    122315.458187
2021-12-06   -236078.855655
2021-12-07   -236789.369380
2021-12-08    121452.597824
2021-12-09    120983.877426
Name: prophet_tuned, dtype: float64